## California Census Data

Build a model of housing prices in the state of California.

*This dataset appeared in a 1997 paper titled Sparse Spatial Autoregressions by Pace, R. Kelley and Ronald Barry, published in the Statistics and Probability Letters journal. They built it using the 1990 California census data. It contains one row per census block group. A block group is the smallest geographical unit for which the U.S. Census Bureau publishes sample data (a block group typically has a population of 600 to 3,000 people).*

Statlib is closed now. But you can get the raw data from [Harry Wang](https://www.kaggle.com/datasets/harrywang/housing) on [Kaggle](https://www.kaggle.com/)